In [ ]:
import importlib

package = "tensorflow"
try:
    importlib.import_module(package)
except ImportError:
    !pip install - U {package}
    importlib.import_module(package)
package = "transformers"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "pandas"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "sklearn"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "keras"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "torch"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "transformers"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "langdetect"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

from google.colab import drive
drive.mount('/content/drive')

import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import classification_report
import pandas as pd

import re
from langdetect import detect

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Zero_Shot_Data_Generation_and_BERT_Roberta_CNN_Curie/news_200.csv')
df1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Zero_Shot_Data_Generation_and_BERT_Roberta_CNN_Curie/generated_combined_data200text-curie-001.csv")


df = df.dropna(subset=['text'])
df = df.dropna(subset=['title'])

df1 = df1.dropna(subset=['text'])
df1 = df1.dropna(subset=['title'])

# Step 3: BERT Tokenization & Formatting
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

def tokenize_text(dfx, max_len):
    return tokenizer.batch_encode_plus(
        dfx['text'].tolist(),
        max_length = max_len,
        pad_to_max_length=True,
        truncation=True,
        return_token_type_ids=False
    )

# Tokenize the text
max_len = 256  # choose a max length
tokens = tokenize_text(df, max_len)
tokens1 = tokenize_text(df1, max_len)

# Step 4: Model Creation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=2,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model = model.to(device)

# Split into training and testing datasets
train_inputs, test_inputs, train_labels, test_labels = train_test_split(tokens['input_ids'], df['label'].values, random_state=100, test_size=0.2)

# Split into training and testing datasets
train_inputs1, test_inputs1, train_labels1, test_labels1 = train_test_split(tokens1['input_ids'], df1['label'].values, random_state=100, test_size=0.2)

train_inputs, train_labels = train_inputs1, train_labels1


# Convert into torch tensors
train_inputs = torch.tensor(train_inputs)
test_inputs = torch.tensor(test_inputs)
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)
# Create DataLoader for the training set
train_data = TensorDataset(train_inputs, train_labels)
train_dataloader = DataLoader(train_data, batch_size=32)

# Step 5: Training
#optimizer = AdamW(model.parameters(), lr=1e-5)
"""
optimizer = AdamW(model.parameters(), lr=1e-5)  # Add weight decay parameter


for epoch in range(1):
    model.train()
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_labels = batch[1].to(device)

        outputs = model(b_input_ids, token_type_ids=None, labels=b_labels)

        loss = outputs[0]
        loss.backward()

        optimizer.step()

"""
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)  # Add weight decay parameter

for epoch in range(3):
    model.train()
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_labels = batch[1].to(device)

        optimizer.zero_grad()

        outputs = model(b_input_ids, token_type_ids=None, labels=b_labels)

        loss = outputs[0]

        # Apply weight decay
        #l2_regularization = torch.tensor(0.)
        l2_regularization = torch.tensor(0., device=device)
        for param in model.parameters():
            l2_regularization += torch.norm(param, p=2)


        loss += 0.01 * l2_regularization  # Adjust the weight decay factor

        loss.backward()
        optimizer.step()

from sklearn.metrics import classification_report
import numpy as np

# Create DataLoader for the test set
test_data = TensorDataset(test_inputs, test_labels)
test_dataloader = DataLoader(test_data, batch_size=32)

# Step 6: Evaluation
model.eval()

predictions = []
true_labels = []

for batch in test_dataloader:
    batch = [b.to(device) for b in batch]
    b_input_ids, b_labels = batch

    with torch.no_grad():
        outputs = model(b_input_ids)

    # Move logits and labels to CPU
    logits = outputs[0].detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)

# Flatten the predictions and true values for aggregate evaluation on the whole dataset
predictions = np.concatenate(predictions, axis=0)
true_labels = np.concatenate(true_labels, axis=0)

# For each input batch, pick the label (0 or 1) with the higher score
pred_flat = np.argmax(predictions, axis=1).flatten()

# Print the classification report
print(classification_report(true_labels, pred_flat, target_names=['Fake News', 'Real News']))


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=fba7727262e86b27ba84371da253062d17e5bbb35e7926184e6723796f4461c7
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


              precision    recall  f1-score   support

   Fake News       0.00      0.00      0.00        19
   Real News       0.51      1.00      0.68        20

    accuracy                           0.51        39
   macro avg       0.26      0.50      0.34        39
weighted avg       0.26      0.51      0.35        39



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import importlib

package = "tensorflow"
try:
    importlib.import_module(package)
except ImportError:
    !pip install - U {package}
    importlib.import_module(package)
package = "transformers"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "pandas"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "sklearn"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "keras"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "torch"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "transformers"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "langdetect"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

from google.colab import drive
drive.mount('/content/drive')

import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import classification_report
import pandas as pd

import re
from langdetect import detect

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Zero_Shot_Data_Generation_and_BERT_Roberta_CNN_Curie/news_500.csv')
df1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Zero_Shot_Data_Generation_and_BERT_Roberta_CNN_Curie/generated_combined_data500text-curie-001-3.csv")

df = df.dropna(subset=['text'])
df = df.dropna(subset=['title'])

df1 = df1.dropna(subset=['text'])
df1 = df1.dropna(subset=['title'])

# Step 3: BERT Tokenization & Formatting
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

def tokenize_text(dfx, max_len):
    return tokenizer.batch_encode_plus(
        dfx['text'].tolist(),
        max_length = max_len,
        pad_to_max_length=True,
        truncation=True,
        return_token_type_ids=False
    )

# Tokenize the text
max_len = 256  # choose a max length
tokens = tokenize_text(df, max_len)
tokens1 = tokenize_text(df1, max_len)

# Step 4: Model Creation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=2,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model = model.to(device)

# Split into training and testing datasets
train_inputs, test_inputs, train_labels, test_labels = train_test_split(tokens['input_ids'], df['label'].values, random_state=100, test_size=0.2)

# Split into training and testing datasets
train_inputs1, test_inputs1, train_labels1, test_labels1 = train_test_split(tokens1['input_ids'], df1['label'].values, random_state=100, test_size=0.2)

train_inputs, train_labels = train_inputs1, train_labels1


# Convert into torch tensors
train_inputs = torch.tensor(train_inputs)
test_inputs = torch.tensor(test_inputs)
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)
# Create DataLoader for the training set
train_data = TensorDataset(train_inputs, train_labels)
train_dataloader = DataLoader(train_data, batch_size=32)

# Step 5: Training
#optimizer = AdamW(model.parameters(), lr=1e-5)
"""
optimizer = AdamW(model.parameters(), lr=1e-5)  # Add weight decay parameter


for epoch in range(1):
    model.train()
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_labels = batch[1].to(device)

        outputs = model(b_input_ids, token_type_ids=None, labels=b_labels)

        loss = outputs[0]
        loss.backward()

        optimizer.step()

"""
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)  # Add weight decay parameter

for epoch in range(3):
    model.train()
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_labels = batch[1].to(device)

        optimizer.zero_grad()

        outputs = model(b_input_ids, token_type_ids=None, labels=b_labels)

        loss = outputs[0]

        # Apply weight decay
        #l2_regularization = torch.tensor(0.)
        l2_regularization = torch.tensor(0., device=device)
        for param in model.parameters():
            l2_regularization += torch.norm(param, p=2)


        loss += 0.01 * l2_regularization  # Adjust the weight decay factor

        loss.backward()
        optimizer.step()

from sklearn.metrics import classification_report
import numpy as np

# Create DataLoader for the test set
test_data = TensorDataset(test_inputs, test_labels)
test_dataloader = DataLoader(test_data, batch_size=32)

# Step 6: Evaluation
model.eval()

predictions = []
true_labels = []

for batch in test_dataloader:
    batch = [b.to(device) for b in batch]
    b_input_ids, b_labels = batch

    with torch.no_grad():
        outputs = model(b_input_ids)

    # Move logits and labels to CPU
    logits = outputs[0].detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)

# Flatten the predictions and true values for aggregate evaluation on the whole dataset
predictions = np.concatenate(predictions, axis=0)
true_labels = np.concatenate(true_labels, axis=0)

# For each input batch, pick the label (0 or 1) with the higher score
pred_flat = np.argmax(predictions, axis=1).flatten()

# Print the classification report
print(classification_report(true_labels, pred_flat, target_names=['Fake News', 'Real News']))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use th

              precision    recall  f1-score   support

   Fake News       0.58      1.00      0.73        54
   Real News       1.00      0.13      0.24        45

    accuracy                           0.61        99
   macro avg       0.79      0.57      0.48        99
weighted avg       0.77      0.61      0.51        99



In [1]:
import importlib

package = "tensorflow"
try:
    importlib.import_module(package)
except ImportError:
    !pip install - U {package}
    importlib.import_module(package)
package = "transformers"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "pandas"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "sklearn"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "keras"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "torch"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "transformers"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "langdetect"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

from google.colab import drive
drive.mount('/content/drive')

import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import classification_report
import pandas as pd

import re
from langdetect import detect

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Zero_Shot_Data_Generation_and_BERT_Roberta_CNN_Curie/news_10000.csv')
df1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Zero_Shot_Data_Generation_and_BERT_Roberta_CNN_Curie/generated_combined_data10000text-curie-001.csv")


df = df.dropna(subset=['text'])
df = df.dropna(subset=['title'])

df1 = df1.dropna(subset=['text'])
df1 = df1.dropna(subset=['title'])

# Step 3: BERT Tokenization & Formatting
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

def tokenize_text(dfx, max_len):
    return tokenizer.batch_encode_plus(
        dfx['text'].tolist(),
        max_length = max_len,
        pad_to_max_length=True,
        truncation=True,
        return_token_type_ids=False
    )

# Tokenize the text
max_len = 256  # choose a max length
tokens = tokenize_text(df, max_len)
tokens1 = tokenize_text(df1, max_len)

# Step 4: Model Creation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=2,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model = model.to(device)

# Split into training and testing datasets
train_inputs, test_inputs, train_labels, test_labels = train_test_split(tokens['input_ids'], df['label'].values, random_state=100, test_size=0.2)

# Split into training and testing datasets
train_inputs1, test_inputs1, train_labels1, test_labels1 = train_test_split(tokens1['input_ids'], df1['label'].values, random_state=100, test_size=0.2)

train_inputs, train_labels = train_inputs1, train_labels1


# Convert into torch tensors
train_inputs = torch.tensor(train_inputs)
test_inputs = torch.tensor(test_inputs)
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)
# Create DataLoader for the training set
train_data = TensorDataset(train_inputs, train_labels)
train_dataloader = DataLoader(train_data, batch_size=32)

# Step 5: Training
#optimizer = AdamW(model.parameters(), lr=1e-5)
"""
optimizer = AdamW(model.parameters(), lr=1e-5)  # Add weight decay parameter


for epoch in range(1):
    model.train()
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_labels = batch[1].to(device)

        outputs = model(b_input_ids, token_type_ids=None, labels=b_labels)

        loss = outputs[0]
        loss.backward()

        optimizer.step()

"""
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)  # Add weight decay parameter

for epoch in range(3):
    model.train()
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_labels = batch[1].to(device)

        optimizer.zero_grad()

        outputs = model(b_input_ids, token_type_ids=None, labels=b_labels)

        loss = outputs[0]

        # Apply weight decay
        #l2_regularization = torch.tensor(0.)
        l2_regularization = torch.tensor(0., device=device)
        for param in model.parameters():
            l2_regularization += torch.norm(param, p=2)


        loss += 0.01 * l2_regularization  # Adjust the weight decay factor

        loss.backward()
        optimizer.step()

from sklearn.metrics import classification_report
import numpy as np

# Create DataLoader for the test set
test_data = TensorDataset(test_inputs, test_labels)
test_dataloader = DataLoader(test_data, batch_size=32)

# Step 6: Evaluation
model.eval()

predictions = []
true_labels = []

for batch in test_dataloader:
    batch = [b.to(device) for b in batch]
    b_input_ids, b_labels = batch

    with torch.no_grad():
        outputs = model(b_input_ids)

    # Move logits and labels to CPU
    logits = outputs[0].detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)

# Flatten the predictions and true values for aggregate evaluation on the whole dataset
predictions = np.concatenate(predictions, axis=0)
true_labels = np.concatenate(true_labels, axis=0)

# For each input batch, pick the label (0 or 1) with the higher score
pred_flat = np.argmax(predictions, axis=1).flatten()

# Print the classification report
print(classification_report(true_labels, pred_flat, target_names=['Fake News', 'Real News']))


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=0ca6bba282344e2279bba182c767f4066016b9b052e0d4613be7454c77fda9be
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


              precision    recall  f1-score   support

   Fake News       0.54      0.99      0.70      1029
   Real News       0.88      0.06      0.12       912

    accuracy                           0.56      1941
   macro avg       0.71      0.53      0.41      1941
weighted avg       0.70      0.56      0.43      1941



In [ ]:
import importlib

package = "tensorflow"
try:
    importlib.import_module(package)
except ImportError:
    !pip install - U {package}
    importlib.import_module(package)
package = "transformers"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "pandas"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "sklearn"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "keras"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "torch"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "transformers"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "langdetect"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

from google.colab import drive
drive.mount('/content/drive')

import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import classification_report
import pandas as pd

import re
from langdetect import detect


df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Zero_Shot_Data_Generation_and_BERT_Roberta_CNN_Curie/train.csv')
df1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Zero_Shot_Data_Generation_and_BERT_Roberta_CNN_Curie/train_curie_20400.csv")


df = df.dropna(subset=['text'])
df = df.dropna(subset=['title'])

df1 = df1.dropna(subset=['text'])
df1 = df1.dropna(subset=['title'])

# Step 3: BERT Tokenization & Formatting
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

def tokenize_text(dfx, max_len):
    return tokenizer.batch_encode_plus(
        dfx['text'].tolist(),
        max_length = max_len,
        pad_to_max_length=True,
        truncation=True,
        return_token_type_ids=False
    )

# Tokenize the text
max_len = 256  # choose a max length
tokens = tokenize_text(df, max_len)
tokens1 = tokenize_text(df1, max_len)

# Step 4: Model Creation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=2,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model = model.to(device)

# Split into training and testing datasets
train_inputs, test_inputs, train_labels, test_labels = train_test_split(tokens['input_ids'], df['label'].values, random_state=100, test_size=0.2)

# Split into training and testing datasets
train_inputs1, test_inputs1, train_labels1, test_labels1 = train_test_split(tokens1['input_ids'], df1['label'].values, random_state=100, test_size=0.2)

train_inputs, train_labels = train_inputs1, train_labels1


# Convert into torch tensors
train_inputs = torch.tensor(train_inputs)
test_inputs = torch.tensor(test_inputs)
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)
# Create DataLoader for the training set
train_data = TensorDataset(train_inputs, train_labels)
train_dataloader = DataLoader(train_data, batch_size=32)

# Step 5: Training
#optimizer = AdamW(model.parameters(), lr=1e-5)
"""
optimizer = AdamW(model.parameters(), lr=1e-5)  # Add weight decay parameter


for epoch in range(1):
    model.train()
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_labels = batch[1].to(device)

        outputs = model(b_input_ids, token_type_ids=None, labels=b_labels)

        loss = outputs[0]
        loss.backward()

        optimizer.step()

"""
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)  # Add weight decay parameter

for epoch in range(3):
    model.train()
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_labels = batch[1].to(device)

        optimizer.zero_grad()

        outputs = model(b_input_ids, token_type_ids=None, labels=b_labels)

        loss = outputs[0]

        # Apply weight decay
        #l2_regularization = torch.tensor(0.)
        l2_regularization = torch.tensor(0., device=device)
        for param in model.parameters():
            l2_regularization += torch.norm(param, p=2)


        loss += 0.01 * l2_regularization  # Adjust the weight decay factor

        loss.backward()
        optimizer.step()

from sklearn.metrics import classification_report
import numpy as np

# Create DataLoader for the test set
test_data = TensorDataset(test_inputs, test_labels)
test_dataloader = DataLoader(test_data, batch_size=32)

# Step 6: Evaluation
model.eval()

predictions = []
true_labels = []

for batch in test_dataloader:
    batch = [b.to(device) for b in batch]
    b_input_ids, b_labels = batch

    with torch.no_grad():
        outputs = model(b_input_ids)

    # Move logits and labels to CPU
    logits = outputs[0].detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)

# Flatten the predictions and true values for aggregate evaluation on the whole dataset
predictions = np.concatenate(predictions, axis=0)
true_labels = np.concatenate(true_labels, axis=0)

# For each input batch, pick the label (0 or 1) with the higher score
pred_flat = np.argmax(predictions, axis=1).flatten()

# Print the classification report
print(classification_report(true_labels, pred_flat, target_names=['Fake News', 'Real News']))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use th

              precision    recall  f1-score   support

   Fake News       0.51      0.74      0.60      2104
   Real News       0.45      0.23      0.30      1937

    accuracy                           0.49      4041
   macro avg       0.48      0.48      0.45      4041
weighted avg       0.48      0.49      0.46      4041

